# Combining Prognostic Models

This section demonstrates how prognostic models can be combined. There are two times in which this is useful: 

1. When combining multiple models of different inter-related systems into one system-of-system model (i.e., [Composite Models](https://nasa.github.io/progpy/api_ref/prog_models/CompositeModel.html)), or
2. Combining multiple models of the same system to be simulated together and aggregated (i.e., [Ensemble Models](https://nasa.github.io/progpy/api_ref/prog_models/EnsembleModel.html)). This is generally done to improve the accuracy of prediction when you have multiple models that each represent part of the behavior or represent a distribution of different behaviors. 

These two methods for combining models are described in the following sections.

## Composite Model

A CompositeModel is a PrognosticsModel that is composed of multiple PrognosticsModels. This is a tool for modeling system-of-systems. i.e., interconnected systems, where the behavior and state of one system affects the state of another system. The composite prognostics models are connected using defined connections between the output or state of one model, and the input of another model. The resulting CompositeModel behaves as a single model.

To illustrate this, we will create a composite model of an aircraft's electric powertrain, combining the DCMotor, ESC, and PropellerLoad models. The Electronic Speed Controller (ESC) converts a commanded duty (i.e., throttle) to signals to the motor. The motor then acts on the signals from the ESC to spin the load, which enacts a torque on the motor (in this case from air resistence).

First we will import the used models, and the CompositeModel class

In [ ]:
from progpy.models import DCMotor, ESC, PropellerLoad
from progpy import CompositeModel

Next we will initiate objects of the individual models that will later create the composite powertrain model.

In [ ]:
m_motor = DCMotor()
m_esc = ESC()
m_load = PropellerLoad()

Next we have to define the connections between the systems. Let's first define the connections from the DCMotor to the propeller load. For this, we'll need to look at the DCMotor states and understand how they influence the PropellerLoad inputs.

In [ ]:
print('motor states: ', m_motor.states)
print('load inputs: ', m_load.inputs)

Each of the states and inputs are described in the model documentation at [DC Motor Docs](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#dc-motor) and [Propeller Docs](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#propellerload)

From reading the documentation we understand that the propeller's velocity is from the motor, so we can define the first connection:

In [ ]:
connections = [
    ('DCMotor.v_rot', 'PropellerLoad.v_rot')
]

Connections are defined as couples where the first value is the input for the second value. The connection above tells the composite model to feed the DCMotor's v_rot into the PropellerLoad's input v_rot.

Next, let's look at the connections the other direction, from the load to the motor.

In [ ]:
print('load states: ', m_load.states)
print('motor inputs: ', m_motor.inputs)

We know here that the load on the motor is from the propeller load, so we can add that connection. 

In [ ]:
connections.append(('PropellerLoad.t_l', 'DCMotor.t_l'))

Now we will repeat the exercise with the DCMotor and ESC.

In [ ]:
print('ESC states: ', m_esc.states)
print('motor inputs: ', m_motor.inputs)
connections.append(('ESC.v_a', 'DCMotor.v_a'))
connections.append(('ESC.v_b', 'DCMotor.v_b'))
connections.append(('ESC.v_c', 'DCMotor.v_c'))

print('motor states: ', m_motor.states)
print('ESC inputs: ', m_esc.inputs)
connections.append(('DCMotor.theta', 'ESC.theta'))

Now we are ready to combine the models. We create a composite model with the inidividual models and the defined connections.

In [ ]:
m_powertrain = CompositeModel(
        (m_esc, m_load, m_motor), 
        connections=connections)

The resulting model includes two inputs, ESC voltage (from the battery) and duty (i.e., commanded throttle). These are the only two inputs not connected internally from the original three models. The states are a combination of all the states of every system. Finally, the outputs are a combination of all the outputs from each of the individual systems. 

In [ ]:
print('inputs: ', m_powertrain.inputs)
print('states: ', m_powertrain.states)
print('outputs: ', m_powertrain.outputs)

Frequently users only want a subset of the outputs from the original model. For example, in this case you're unlikely to be measuring the individual voltages from the ESC. Outputs can be specified when creating the composite model. For example:

In [ ]:
m_powertrain = CompositeModel(
        (m_esc, m_load, m_motor), 
        connections=connections,
        outputs={'DCMotor.v_rot', 'DCMotor.theta'})
print('outputs: ', m_powertrain.outputs)

Now the outputs are only DCMotor angle and velocity.

The resulting model can be used in simulation, state estimation, and prediction the same way any other model would be, as demonstrated below:

In [ ]:
load = m_powertrain.InputContainer({
        'ESC.duty': 1, # 100% Throttle
        'ESC.v': 23
    })
def future_loading(t, x=None):
    return load

simulated_results = m_powertrain.simulate_to(2, future_loading, dt=2.5e-5, save_freq=1e-2)
fig = simulated_results.outputs.plot(compact=False, keys=['DCMotor.v_rot'], ylabel='Velocity')
fig = simulated_results.states.plot(keys=['DCMotor.i_b', 'DCMotor.i_c', 'DCMotor.i_a'], ylabel='ESC Currents')

## Ensemble Model

An ensemble model is an approach to modeling where one or more models of the same system are simulated together and then aggregated into a single prediction. This can be multiple versions of the same model with different parameters, or different models of the same system representing different parts of the system's behavior. This is generally done to improve the accuracy of prediction when you have multiple models that each represent part of the behavior or represent a distribution of different behaviors.

In ensemble models, aggregation occurs in two steps, at state transition and then output, event state, threshold met, or performance metric calculation. At each state transition, the states from each aggregate model are combined based on the defined aggregation method. When calling output, the resulting outputs from each aggregate model are similarily combined. The default method is mean, but the user can also choose to use a custom aggregator.

![Aggregation](img/aggregation.png)

To illustrate this, let's create an example where there we have four equivalent circuit models, each with different configuration parameters, below. These represent the range of possible configurations expected for our example system.

In [ ]:
from progpy.models import BatteryCircuit
m_circuit = BatteryCircuit()
m_circuit_2 = BatteryCircuit(qMax = 7860)
m_circuit_3 = BatteryCircuit(qMax = 6700, Rs = 0.055)

Let's create an EnsembleModel which combines each of these.

In [ ]:
from progpy import EnsembleModel
m_ensemble = EnsembleModel(
    models=(m_circuit, m_circuit_2, m_circuit_3))

Now let's evaluate the performance of the combined model using real battery data from NASA's prognostic data repository. See 07. Datasets for more detail on accessing data from this repository

In [ ]:
from progpy.datasets import nasa_battery
data = nasa_battery.load_data(batt_id=8)[1]
RUN_ID = 0
test_input = [{'i': i} for i in data[RUN_ID]['current']]
test_time = data[RUN_ID]['relativeTime']

To evaluate the model we first create a future loading function that uses the loading from the data.

In [ ]:
def future_loading(t, x=None):
    for i, mission_time in enumerate(test_time):
        if mission_time > t:
            return m_circuit.InputContainer(test_input[i])

Next we will simulate the ensemble model

In [ ]:
t_end = test_time.iloc[-1]
results_ensemble = m_ensemble.simulate_to(t_end, future_loading)

Finally, we compare the voltage predicted by the ensemble model with the ground truth from dataset.

In [ ]:
from matplotlib import pyplot as plt
fig = plt.plot(test_time, data[RUN_ID]['voltage'], color='green', label='ground truth')
fig = plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='red', label='ensemble')
plt.xlabel('Time (s)')
plt.ylabel('Voltage')
plt.legend()

The ensemble model actually performs pretty poorly here. This is mostly because there's an outlier model (m_circuit_3). This can be resolved using a different aggregation method. By default, aggregation uses the mean. Let's update the ensemble model to use median and resimulate

In [ ]:
import numpy as np
m_ensemble.parameters['aggregation_method'] = np.median

results_ensemble_median = m_ensemble.simulate_to(t_end, future_loading)
fig = plt.plot(results_ensemble_median.times, [z['v'] for z in results_ensemble_median.outputs], color='orange', label='ensemble -median')
fig = plt.plot(test_time, data[RUN_ID]['voltage'], color='green', label='ground truth')
fig = plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='red', label='ensemble')
plt.xlabel('Time (s)')
plt.ylabel('Voltage')
plt.legend()

Much better!

The same ensemble approach can be used with a heterogeneous set of models that have different states.

Here we will repeat the exercise using the battery electrochemisty and equivalent circuit models. The two models share one state in common (tb), but otherwise are different

In [ ]:
from progpy.models import BatteryElectroChemEOD
m_electro = BatteryElectroChemEOD(qMobile=7800)

print('Electrochem states: ', m_electro.states)
print('Equivalent Circuit States', m_circuit.states)

Now let's create an ensemble model combining these and evaluate it.

In [ ]:
m_ensemble = EnsembleModel((m_circuit, m_electro))
results_ensemble = m_ensemble.simulate_to(t_end, future_loading)

To compare these results, let's also simulate the two models that comprise the ensemble model.

In [ ]:
results_circuit1 = m_circuit.simulate_to(t_end, future_loading)
results_electro = m_electro.simulate_to(t_end, future_loading)

The results of each of these are plotted below.

In [ ]:
plt.figure()
plt.plot(results_circuit1.times, [z['v'] for z in results_circuit1.outputs], color='blue', label='circuit')
plt.plot(results_electro.times, [z['v'] for z in results_electro.outputs], color='red', label='electro chemistry')
plt.plot(results_ensemble.times, [z['v'] for z in results_ensemble.outputs], color='yellow', label='ensemble')
plt.plot(test_time, data[RUN_ID]['voltage'], color='green', label='ground truth')
plt.legend()

Note that the result may not be exactly between the other two models. This is because of aggregation is done in 2 steps: at state transition and then at output calculation.

Ensemble models can be further extended to include an aggregator that selects the best model at any given time. That feature is described in the following section.

## Mixture of Experts (MoE)